<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/final/single_cls_gpt4o_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# OpenAI 패키지 설치
!pip install openai

In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import os
from datasets import Dataset, load_dataset, DatasetDict
import ast
import evaluate
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
classes = ['biology', 'cooking', 'diy', 'travel', 'stackoverflow']
class2id = {'biology' :0, 'cooking' : 1, 'diy' : 2, 'travel' : 3, 'stackoverflow' : 4}
id2class = {0 : 'biology', 1: 'cooking', 2 : 'diy', 3 : 'travel', 4 : 'stackoverflow'}

In [ ]:
import pandas as pd


valid_df = pd.read_csv("/content/drive/My Drive/AiExpertCource/project/dataset/stackexchage/stackexchange_test_dataset_1000_title.csv")

In [ ]:
len(valid_df)

1000

In [ ]:
valid_df.head()

,title_content,label,Tags_new,Algorithms,Backend,Data Science,Databases,Dev Tools,Frontend,Mobile,Systems,iOS/macOS
0,0.1 M sodium citrate in 10% ethanol and DNA so...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fatty Acid Synthesis,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Is the autonomic nervous system only activated...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Which are the last cells of the human body to ...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Did vertebrates diverge from other deuterostom...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
selected_columns = ['title_content', 'label']  # 원하는 컬럼 이름 리스트
valid_df = valid_df[selected_columns]

In [ ]:
valid_df.head()

,title_content,label
0,0.1 M sodium citrate in 10% ethanol and DNA so...,0
1,Fatty Acid Synthesis,0
2,Is the autonomic nervous system only activated...,0
3,Which are the last cells of the human body to ...,0
4,Did vertebrates diverge from other deuterostom...,0


In [ ]:
from openai import OpenAI
import os
from getpass import getpass

In [ ]:
os.environ['OPENAI_API_KEY'] = 'aa'
system_input = "You are a classifier that categorizes text into one of 5 labels. The label list is as follows: [biology, cooking, diy, travel, stackoverflow]. You must choose only one label. Provide your answer in the format label:"

def call_api(user_input, model="gpt-4o-mini", temperature=0, max_tokens=30):
    try:
        # 메시지 목록을 자동으로 생성해요
        messages = [
            {"role": "system", "content": system_input},
            {"role": "user", "content": user_input}
        ]

        response = OpenAI().chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens  # 최대 토큰 수를 지정해요
        )
        # 생성된 응답을 반환해요
        return response
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
counter = [0]

In [ ]:
import re

In [ ]:
def predict(test):

  prompt = test["title_content"]
  response = call_api(prompt)
  answer = response.choices[0].message.content.split("label:")[-1].strip()
  cleaned_text = re.sub(r'[^a-zA-Z]', '', answer)
  try:
    test['predict'] = class2id[cleaned_text]
  except KeyError:
    print(f"Warning: Unexpected label '{answer}' encountered. Skipping this row.")
    test['predict'] = -1

  if counter[0] % 100 == 0:
        print(f"Processing row {counter[0]}")
  counter[0] += 1

  return test


In [ ]:
valid_df = valid_df.apply(predict, axis=1)

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900


In [ ]:
valid_df.head()

,title_content,label,predict
0,0.1 M sodium citrate in 10% ethanol and DNA so...,0,0
1,Fatty Acid Synthesis,0,0
2,Is the autonomic nervous system only activated...,0,0
3,Which are the last cells of the human body to ...,0,0
4,Did vertebrates diverge from other deuterostom...,0,0


In [ ]:
y_pred = np.array([item['predict'] for item in valid_df.to_dict('records')])  # Convert DataFrame to list of dictionaries
y_true = np.array([item['label'] for item in valid_df.to_dict('records')])  # Convert DataFrame to list of dictionaries

In [ ]:
print(compute_metrics(y_pred, y_true))

{'accuracy': 0.907, 'precision': 0.907, 'recall': 0.907, 'f1': 0.907}


In [5]:
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(predictions, labels):

    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "micro_precision": precision.compute(predictions=predictions, references=labels, average="micro")["precision"],
        "micro_recall": recall.compute(predictions=predictions, references=labels, average="micro")["recall"],
        "micro_f1": f1.compute(predictions=predictions, references=labels, average="micro")["f1"],
        "macro_precision": precision.compute(predictions=predictions, references=labels, average="macro")["precision"],
        "macro_recall": recall.compute(predictions=predictions, references=labels, average="macro")["recall"],
        "macro_f1": f1.compute(predictions=predictions, references=labels, average="macro")["f1"],
        "weighted_precision": precision.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "weighted_recall": recall.compute(predictions=predictions, references=labels, average="weighted")["recall"],
        "weighted_f1": f1.compute(predictions=predictions, references=labels, average="weighted")["f1"],
    }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
valid_df.to_csv('/content/drive/My Drive/AiExpertCource/project/dataset/stackexchage/stackexchange_test_dataset_1000_title_predict.csv', index=False)

NameError: name 'valid_df' is not defined

In [6]:
import pandas as pd


valid_df = pd.read_csv("/content/drive/My Drive/AiExpertCource/project/dataset/stackexchage/stackexchange_test_dataset_1000_title_predict.csv")

In [7]:
y_pred = np.array([item['predict'] for item in valid_df.to_dict('records')])  # Convert DataFrame to list of dictionaries
y_true = np.array([item['label'] for item in valid_df.to_dict('records')])  # Convert DataFrame to list of dictionaries

In [8]:
print(compute_metrics(y_pred, y_true))

{'accuracy': 0.907, 'micro_precision': 0.907, 'micro_recall': 0.907, 'micro_f1': 0.907, 'macro_precision': 0.9196183602488504, 'macro_recall': 0.907, 'macro_f1': 0.9072543651815057, 'weighted_precision': 0.9196183602488505, 'weighted_recall': 0.907, 'weighted_f1': 0.9072543651815057}
